# 암환자 유전체 데이터 기반 암종 분류 AI 모델 개발


- '2024 생명연구자원 AI활용 경진대회'는 바이오 데이터를 기반으로 한 AI 기술의 문제 해결 능력을 탐구하는 것을 목표로 합니다. <br>이 대회는 바이오 분야에서 AI 활용의 저변을 확대하고, 복잡한 바이오 데이터를 효율적으로 분석 및 해석할 수 있는 AI 알고리즘 개발에 초점을 맞추고 있습니다. <br><br>
- 본 대회의 구체적인 과제는 암환자 유전체 데이터의 변이 정보를 활용하여 암종을 분류하는 AI 모델을 개발하는 것입니다. <br>참가자들은 제공된 학습 데이터셋(암환자 유전체 변이 정보)을 사용하여 특정 변이 정보를 바탕으로 암종을 정확하게 분류할 수 있는 AI 알고리즘을 개발해야 합니다. <br><br>
- 이 대회의 궁극적인 목적은 바이오 데이터의 활용도를 높이고, 바이오 분야에서 AI 기술의 적용 가능성을 극대화하며, 인공지능 기술이 실제 바이오 의료 문제 해결에 어떻게 기여할 수 있는지 탐구하는 것입니다.

# Import library

In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix
# import xgboost as xgb

In [21]:
# 평가용 함수
def print_score(clf, x_train,y_train,x_test,y_test, train=True):
    if train:
        pred = clf.predict(x_train)
        clf_report = pd.DataFrame(classification_report(y_train,pred,output_dict=True,zero_division=0))
        print("Train Result : \n ==============================================")
        print(f"Accuracy score : {accuracy_score(y_train,pred)*100:.2f}%")
        print("---------------------------------------------------------------")
        print(f"Calssfication Report:\n{clf_report}")
        print('---------------------------------------------------------------')
        print(f'Confusion Matrix: \n{confusion_matrix(y_train,pred)}\n')
    else:
        pred = clf.predict(x_test)
        clf_report = pd.DataFrame(classification_report(y_test,pred,output_dict=True,zero_division=0))
        print("Test Result : \n ==============================================")
        print(f"Accuracy score : {accuracy_score(y_test,pred)*100:.2f}%")
        print("---------------------------------------------------------------")
        print(f"Calssfication Report:\n{clf_report}")
        print('---------------------------------------------------------------')
        print(f'Confusion Matrix: \n{confusion_matrix(y_test,pred)}\n')

# Load Data

In [4]:
mainPath = 'C:/pandas/암환자 유전체 데이터 기반 암종 분류 AI 모델_data'
train = pd.read_csv(mainPath + "/train.csv")
test = pd.read_csv(mainPath + "/test.csv")

In [5]:
train.head()

,ID,SUBCLASS,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,TRAIN_0000,KIPAN,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1,TRAIN_0001,SARC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
2,TRAIN_0002,SKCM,R895R,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
3,TRAIN_0003,KIRC,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
4,TRAIN_0004,GBMLGG,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


# Data Preprocessing

In [6]:
# SUBCLASS 가 범주형이기 때문에 LabelEncoder 사용
le_subclass = LabelEncoder()
train['SUBCLASS'] = le_subclass.fit_transform(train['SUBCLASS'])

# 변환된 레이블 확인
# for i, label in enumerate(le_subclass.classes_):
#     print(f"원래 레이블: {label}, 변환된 숫자: {i}")

In [7]:
## x 의 경우도 범주형으로 구성되어 있어, 알맞은 인코딩 필요
X = train.drop(columns=['SUBCLASS', 'ID'])
y_subclass = train['SUBCLASS']

categorical_columns = X.select_dtypes(include=['object', 'category']).columns
ordinal_encoder = OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1)
X_encoded = X.copy()
X_encoded[categorical_columns] = ordinal_encoder.fit_transform(X[categorical_columns])

In [8]:
# X_encoded  범주형 데이터는 순서가 있는 라벨링으로 변환한 학습용 데이터
print(X_encoded.shape, y_subclass.shape)

(6201, 4384) (6201,)


# Model Define and Train

In [28]:
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

x_train,x_test,y_train,y_test = train_test_split(X_encoded,y_subclass,stratify=y_subclass,random_state=42)

pipeline = Pipeline([
    ('std_scaler',StandardScaler()), 
    ('logistic',LogisticRegression(max_iter=10000,C=0.1,penalty='l1',solver = "liblinear" ))
    ])
pipeline.fit(x_train, y_train)

Pipeline(steps=[('std_scaler', StandardScaler()),
                ('logistic',
                 LogisticRegression(C=0.1, max_iter=10000, penalty='l1',
                                    solver='liblinear'))])

In [29]:
print_score(pipeline,x_train,y_train,x_test,y_test,train=True)
print_score(pipeline,x_train,y_train,x_test,y_test,train=False)

Train Result : 
Accuracy score : 76.32%
---------------------------------------------------------------
Calssfication Report:
                   0          1           2           3           4  \
precision   1.000000   0.974026    0.517808    0.979167    0.933333   
recall      0.944444   0.961538    0.962649    0.810345    0.922156   
f1-score    0.971429   0.967742    0.673397    0.886792    0.927711   
support    54.000000  78.000000  589.000000  116.000000  167.000000   

                   5           6           7           8           9  ...  \
precision   0.916667    0.708229    0.953947    0.663113    0.700599  ...   
recall      0.758621    0.820809    0.868263    0.805699    0.468000  ...   
f1-score    0.830189    0.760375    0.909091    0.727485    0.561151  ...   
support    29.000000  346.000000  167.000000  386.000000  250.000000  ...   

                   19          20          21         22          23  \
precision    0.897436    0.994872    0.974265   0.948052    

In [ ]:
#---------------------------------- 모델개발이 완료되면 정답 생성 --------------------

In [14]:
model.fit(X_encoded, y_subclass)

SVC()

In [15]:
test.head()

,ID,A2M,AAAS,AADAT,AARS1,ABAT,ABCA1,ABCA2,ABCA3,ABCA4,...,ZNF292,ZNF365,ZNF639,ZNF707,ZNFX1,ZNRF4,ZPBP,ZW10,ZWINT,ZYX
0,TEST_0000,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
1,TEST_0001,WT,WT,WT,WT,WT,R587Q,WT,WT,WT,...,WT,WT,WT,WT,WT,I383Sfs,WT,WT,WT,WT
2,TEST_0002,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
3,TEST_0003,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT
4,TEST_0004,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT


# Inference

In [16]:
test_X = test.drop(columns=['ID'])
X_encoded = test_X.copy()
X_encoded[categorical_columns] = ordinal_encoder.transform(test_X[categorical_columns])

In [17]:
# 테스트 데이터로 예측
predictions = model.predict(X_encoded)

In [18]:
predictions

array([21, 21,  2, ..., 25,  2,  2])

In [19]:
original_labels = le_subclass.inverse_transform(predictions)
original_labels

array(['STES', 'STES', 'BRCA', ..., 'UCEC', 'BRCA', 'BRCA'], dtype=object)

# Submisson

In [22]:
submisson = pd.read_csv(mainPath+"/sample_submission.csv")
submisson.head()

,ID,SUBCLASS
0,TEST_0000,LGG
1,TEST_0001,LGG
2,TEST_0002,LGG
3,TEST_0003,LGG
4,TEST_0004,LGG


In [23]:
submisson["SUBCLASS"] = original_labels

In [24]:
submisson.to_csv('./baseline_submission.csv', encoding='UTF-8-sig', index=False)